In [1]:
from datetime import datetime
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-03-17 18:26:42.893507


# Install Packages

In [3]:
!pip install albumentations==0.4.5

     |████████████████████████████████| 122kB 10.5MB/s 
     |████████████████████████████████| 634kB 5.0MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=64378 sha256=26c71e19649c5307ac32faf16c053aa8af0b74b1c138c435a65c0825b6f2d422
  Stored in directory: /root/.cache/pip/wheels/f0/a0/61/e50f93165a5ec7e7f5d65064e513239505bc4c06d2289557d3
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=4510fcc6d3ccdf01c1c6d74e3be4d226d472e4edc95dea88099bd41770b6183c
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


# Import Files and Packages

In [0]:
from torchsummary import summary

from cuda import initialize_cuda
from data.dataset import CIFAR10
from model import QuizDNN
from model.utils.loss import cross_entropy_loss
from model.utils.optimizers import sgd_optimizer
from model.utils.callbacks import lr_scheduler
from train import train
from evaluate import val

# Initialize Hyperparameters

In [0]:
class Args:

    # Data Loading
    # ============

    train_batch_size = 64
    val_batch_size = 64
    num_workers = 4

    # Training
    # ========
    random_seed = 1
    epochs = 10
    learning_rate = 0.01
    momentum = 0.9
    sample_count = 10

## Set Seed and Get GPU Availability

In [6]:
# Initialize CUDA and set random seed
cuda, device = initialize_cuda(Args.random_seed)

GPU Available? True


# Download CIFAR-10 Dataset

In [7]:
dataset = CIFAR10(
    train_batch_size=Args.train_batch_size,
    val_batch_size=Args.val_batch_size,
    cuda=cuda,
    num_workers=Args.num_workers
)


Extracting /content/data/cifar10/cifar-10-python.tar.gz to /content/data/cifar10
Files already downloaded and verified
Files already downloaded and verified


## Training and Validation Dataloaders

In [0]:
# Create train data loader
train_loader = dataset.loader(train=True)

# Create val data loader
val_loader = dataset.loader(train=False)

# Model Architecture and Summary

In [9]:
model = QuizDNN().to(device)
summary(model, dataset.image_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
       BatchNorm2d-3           [-1, 64, 32, 32]             128
            Conv2d-4           [-1, 64, 32, 32]          36,928
              ReLU-5           [-1, 64, 32, 32]               0
       BatchNorm2d-6           [-1, 64, 32, 32]             128
            Conv2d-7           [-1, 64, 32, 32]          36,928
              ReLU-8           [-1, 64, 32, 32]               0
       BatchNorm2d-9           [-1, 64, 32, 32]             128
        MaxPool2d-10           [-1, 64, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]          36,928
             ReLU-12           [-1, 64, 16, 16]               0
      BatchNorm2d-13           [-1, 64, 16, 16]             128
           Conv2d-14           [-1, 64,

# Model Training and Validation

In [10]:
losses = []
accuracies = []
correct_samples = []
incorrect_samples = []

criterion = cross_entropy_loss()  # Create loss function
optimizer = sgd_optimizer(model, Args.learning_rate, Args.momentum)  # Create optimizer
last_epoch = False

for epoch in range(1, Args.epochs + 1):
    print(f'Epoch {epoch}:')
    if epoch == Args.epochs:
        last_epoch = True
    train(model, train_loader, device, optimizer, criterion)
    val(
        model, val_loader, device, criterion, losses, accuracies,
        correct_samples, incorrect_samples, Args.sample_count, last_epoch=last_epoch
    )

  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 1:


  0%|          | 0/782 [00:00<?, ?it/s]


Validation set: Average loss: 0.0164, Accuracy: 6222/10000 (62.22%)

Epoch 2:


  0%|          | 0/782 [00:00<?, ?it/s]

...


Validation set: Average loss: 0.0085, Accuracy: 8254/10000 (82.54%)

Epoch 9:


  0%|          | 0/782 [00:00<?, ?it/s]


Validation set: Average loss: 0.0092, Accuracy: 8209/10000 (82.09%)

Epoch 10:


Loss=0.45 Batch_ID=781 Accuracy=93.41: 100%|██████████| 782/782 [00:21<00:00, 37.00it/s]



Validation set: Average loss: 0.0089, Accuracy: 8220/10000 (82.20%)

